In [50]:
import sys
sys.path.append('../lib/')

import pandas as pd
import numpy as np
import geopandas as gpd

import math

import scipy.interpolate
from copy import deepcopy
from shapely.geometry import Point
from collections import defaultdict

In [13]:
grids = gpd.read_file('../results/grids_vgr_1km_density_deso_with_5km_upper_grids.shp')

In [41]:
#Filter out the grids which do not have the density data
deleteIndex = []

within_1 = defaultdict(list)


for index, row in grids.iterrows():
    if row['deso'] == None or math.isnan(row['density']):
        deleteIndex.append(index)
    else:
        within_1[row['deso']].append(row['zone'])
grids = grids.drop(index = deleteIndex)

little_name = list(grids.zone)
population_density = dict(zip(grids.zone, grids.density))
geometry = dict(zip(grids.zone, grids.geometry))


In [42]:
grids.head()

,zone,deso,density,upper_deso,upper_dens,upper_zone,upper_xcoo,upper_ycoo,geometry
0,24,1401A0010,26.061368,1401A0010,26.061368,1,347500.0,6392500.0,"POLYGON ((344000.000 6389000.000, 344000.000 6..."
1,25,1401A0010,26.061368,1401A0010,26.061368,1,347500.0,6392500.0,"POLYGON ((344000.000 6390000.000, 344000.000 6..."
2,26,1401A0010,26.061368,1401A0010,26.061368,1,347500.0,6392500.0,"POLYGON ((344000.000 6391000.000, 344000.000 6..."
3,27,1401A0010,26.061368,1401A0010,26.061368,1,347500.0,6392500.0,"POLYGON ((344000.000 6392000.000, 344000.000 6..."
4,28,1401A0010,26.061368,1401A0010,26.061368,1,347500.0,6392500.0,"POLYGON ((344000.000 6393000.000, 344000.000 6..."


In [18]:
big_grid = grids[["upper_dens", "upper_zone", "upper_xcoo", "upper_ycoo"]].drop_duplicates()

In [33]:
name_big = list(big_grid.upper_zone)
den_big = dict(zip(big_grid.upper_zone, big_grid.upper_dens))
geo_big = dict(zip(big_grid.upper_zone, list(zip(big_grid.upper_xcoo, big_grid.upper_ycoo))))

In [32]:
base_info = gpd.read_file('../dbs/sweden/zones/DeSO/DeSO_2018_v2.shp')
base_info.loc[:, 'deso_3'] = base_info.loc[:, 'deso'].apply(lambda x: x[:2])
zones_subset = base_info.loc[base_info['deso_3'] == '14', :]
zones_subset_info = dict(zip(zones_subset['deso'], zones_subset['geometry']))
zone_name = list(zones_subset['deso'])


In [47]:
cover = []
checked = set()
for i in range(0, len(zone_name)):
    sub_cover = []
    for j in range(0, len(name_big)):
        if j in checked:
            continue
        point_j = Point(geo_big[name_big[j]][0], geo_big[name_big[j]][1])
        if zones_subset_info[zone_name[i]].contains(point_j) == True:
            # grid_j in zone_i
            sub_cover.append(name_big[j])
            checked.add(j) # This grid has been occupied, we do not need to check it again.
    cover.append(sub_cover)
within = dict(zip(zone_name, cover))


bigzone_name = []
bigCover = []
subCover = []
old_name = zone_name[0][0:5]

bigCover_1 = []
subCover_1 = []

bigzone_name.append(old_name)
subCover.extend(within[zone_name[0]])
subCover_1.extend(within_1[zone_name[0]])

for i in range(1, len(zone_name)):
    new_name = zone_name[i][0:5]

    if new_name == old_name:
        # this two zones belong the same big zone
        subCover.extend(within[zone_name[i]])
        subCover_1.extend(within_1[zone_name[i]])

    if new_name != old_name:
        # find a new big zone
        #store old results
        bigCover.append(deepcopy(subCover))
        subCover.clear()

        bigCover_1.append(deepcopy(subCover_1))
        subCover_1.clear()

        #store new resutls
        bigzone_name.append(new_name)
        subCover.extend(within[zone_name[i]])
        subCover_1.extend(within_1[zone_name[i]])

    old_name = new_name

# handle the lastest case
bigCover.append(subCover)
bigCover_1.append(subCover_1)

big_within = dict(zip(bigzone_name, bigCover))
big_within_1 = dict(zip(bigzone_name, bigCover_1))

In [51]:
df_data = pd.read_csv('../results/distance_ratio_data.csv')
df_simulation = pd.read_csv('../results/distance_ratio_simulation.csv')


sp_data_sweden = scipy.interpolate.interp1d(df_data.loc[df_data.country == 'sweden', ['distance']].values.reshape(-1),
                                            df_data.loc[df_data.country == 'sweden', ['ratio']].values.reshape(-1), bounds_error = False, fill_value = 1.5)

sp_data_simulation = scipy.interpolate.interp1d(df_simulation.loc[:, ['distance']].values.reshape(-1),
                                                df_simulation.loc[:, ['ratio']].values.reshape(-1),  bounds_error = False, fill_value = 1.5)

In [53]:
model_output = dict()
demand_d = dict()
demand_D_survey = dict()
demand_D_simulation = dict()


area_1 = 1
r_average_1 = area_1 / math.pi

area_5 = 25
r_average_5= area_5 / math.pi

T = 1000
f_max = 1
f_min = 1/T
parameter = math.log(f_max / f_min)

p_1 = area_1 * r_average_1 * parameter
p_5 = area_5 * r_average_5 * parameter


for i in range(0, len(bigzone_name)):
    element = dict()
    element1 = dict()
    element2 = dict()
    element3 = dict()
    for j in range(i, len(bigzone_name)): 
        average_daily_trips = 0
        demand_d_tmp = 0
        demand_D_tmp = 0
        demand_D_simulation_tmp = 0
        if i == j:
            for begin in range(0, len(big_within_1[bigzone_name[i]])):
                for end in range(0, len(big_within_1[bigzone_name[j]])):
                    if begin != end:
                        deltax = (geometry[big_within_1[bigzone_name[i]][begin]].centroid.x - geometry[big_within_1[bigzone_name[j]][end]].centroid.x) / 1000                    
                        deltay = (geometry[big_within_1[bigzone_name[i]][begin]].centroid.y - geometry[big_within_1[bigzone_name[j]][end]].centroid.y) / 1000
                
                        distance = deltax * deltax + deltay * deltay

                        root_distance = math.sqrt(distance)
                    
                        tmp =  ( population_density[big_within_1[bigzone_name[i]][begin]] + population_density[big_within_1[bigzone_name[j]][end]] ) / distance
                        tmp1 = tmp * root_distance
                        tmp2 = tmp * float(sp_data_sweden(root_distance)) * root_distance
                        tmp3 = tmp * float(sp_data_simulation(root_distance)) * root_distance

                        average_daily_trips = average_daily_trips + tmp
                        demand_d_tmp = demand_d_tmp + tmp1
                        demand_D_tmp = demand_D_tmp + tmp2
                        demand_D_simulation_tmp = demand_D_simulation_tmp + tmp3

            element[bigzone_name[j]] = p_1 * average_daily_trips
            element1[bigzone_name[j]] = p_1 * demand_d_tmp
            element2[bigzone_name[j]] = p_1 * demand_D_tmp
            element3[bigzone_name[j]] = p_1 * demand_D_simulation_tmp
        else:
            for begin in range(0, len(big_within[bigzone_name[i]])):
                for end in range(0, len(big_within[bigzone_name[j]])):
                        deltax = (geo_big[big_within[bigzone_name[i]][begin]][0] - geo_big[big_within[bigzone_name[j]][end]][0]) / 1000                    
                        deltay = (geo_big[big_within[bigzone_name[i]][begin]][1] - geo_big[big_within[bigzone_name[j]][end]][1]) / 1000
                
                        distance = deltax * deltax + deltay * deltay

                        root_distance = math.sqrt(distance)
                    
                        tmp =  ( den_big[big_within[bigzone_name[i]][begin]] + den_big[big_within[bigzone_name[j]][end]] ) / distance
                        tmp1 = tmp * root_distance
                        tmp2 = tmp * float(sp_data_sweden(root_distance)) * root_distance
                        tmp3 = tmp * float(sp_data_simulation(root_distance)) * root_distance

                        average_daily_trips = average_daily_trips + tmp
                        demand_d_tmp = demand_d_tmp + tmp1
                        demand_D_tmp = demand_D_tmp + tmp2
                        demand_D_simulation_tmp = demand_D_simulation_tmp + tmp3

            element[bigzone_name[j]] = p_5 * average_daily_trips
            element1[bigzone_name[j]] = p_5 * demand_d_tmp
            element2[bigzone_name[j]] = p_5 * demand_D_tmp
            element3[bigzone_name[j]] = p_5 * demand_D_simulation_tmp
    model_output[bigzone_name[i]] = element
    demand_d[bigzone_name[i]] = element1
    demand_D_survey[bigzone_name[i]] = element2
    demand_D_simulation[bigzone_name[i]] = element3

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126


In [54]:
results_output = "../results/model_output_5km_1km.txt"
results_demand_d = '../results/demand_d_5km_1km.txt'
results_demand_D_servey = '../results/demand_D_survey_5km_1km.txt'
results_demand_D_simulation = '../results/demand_D_simulation_5km_1km.txt'

In [55]:
def store(results_output, model_output):
    result = np.zeros(len(bigzone_name)*len(bigzone_name))
    for i in range(0, len(bigzone_name)):
        for j in range(0, len(bigzone_name)):
            if i <= j:
                result[i + j * len(bigzone_name)] = model_output[bigzone_name[i]][bigzone_name[j]]
            if i > j:
                result[i + j * len(bigzone_name)] = model_output[bigzone_name[j]][bigzone_name[i]]
    np.savetxt(results_output, result)

In [56]:
store(results_output, model_output)
store(results_demand_d, demand_d)
store(results_demand_D_servey, demand_D_survey)
store(results_demand_D_simulation, demand_D_simulation)